In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim

In [2]:
from mnist import MNIST

In [3]:
mnist_loader = MNIST("/home/cyclops/NEU/CS6220/data/mnist/")
mnist_loader.gz = True

In [4]:
images, labels = mnist_loader.load_training()
timages, tlabels = mnist_loader.load_testing()

In [5]:
images = np.array(images)
labels = np.array(labels)
timages = np.array(timages)
tlabels = np.array(tlabels)

In [6]:
labels = np.reshape(labels, (-1, 1))
tlabels = np.reshape(tlabels, (-1, 1))

In [7]:
X = Variable(torch.from_numpy(images))
y = Variable(torch.from_numpy(labels), requires_grad=False)
_X = Variable(torch.from_numpy(timages))
_y = Variable(torch.from_numpy(tlabels), requires_grad=False)

In [8]:
y = y.type(torch.LongTensor).squeeze_()
_y = _y.type(torch.LongTensor).squeeze_()

In [10]:
X = X.type(torch.FloatTensor)
_X = _X.type(torch.FloatTensor)

In [34]:
class MNISTNET(nn.Module):
    
    def __init__(self):
        super(MNISTNET, self).__init__()
        
        self.layer1 = nn.Linear(28 * 28, 512)
        self.layer2 = nn.Linear(512, 256)
#         self.layer3 = nn.Linear(256, 128)
#         self.layer4 = nn.Linear(128, 32)
        self.out = nn.Linear(256, 10)
        
    def forward(self, x):
        x = self.layer1(x)
        x = F.sigmoid(x)
        x = self.layer2(x)
        x = F.sigmoid(x)
#         x = self.layer3(x)
#         x = F.sigmoid(x)
#         x = self.layer4(x)
#         x = F.sigmoid(x)
        x = self.out(x)
        return F.log_softmax(x)

In [35]:
net = MNISTNET()
net.cuda()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
loss_op = nn.CrossEntropyLoss()

In [36]:
X = X.cuda()
y = y.cuda()

In [37]:
for epoch in range(101):
    
    optimizer.zero_grad()
    out = net(X)

    loss = loss_op(out, y)
    loss.backward()
    optimizer.step()
    
    print(epoch, loss.data.item())

/home/cyclops/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 2.337862491607666
1 2.3316357135772705
2 2.323411464691162
3 2.3153765201568604
4 2.3083691596984863
5 2.3025426864624023
6 2.2977633476257324
7 2.2938170433044434
8 2.2904975414276123
9 2.2876343727111816
10 2.2850961685180664
11 2.2827870845794678
12 2.280637741088867
13 2.2786014080047607
14 2.276644229888916
15 2.274742364883423
16 2.272878885269165
17 2.2710413932800293
18 2.269221544265747
19 2.2674148082733154
20 2.265618085861206
21 2.2638258934020996
22 2.262038230895996
23 2.260254144668579
24 2.2584729194641113
25 2.2566938400268555
26 2.2549166679382324
27 2.253139019012451
28 2.2513628005981445
29 2.249584913253784
30 2.247807025909424
31 2.2460291385650635
32 2.2442519664764404
33 2.242475748062134
34 2.240699052810669
35 2.238922119140625
36 2.2371444702148438
37 2.2353668212890625
38 2.2335877418518066
39 2.2318074703216553
40 2.230025291442871
41 2.2282416820526123
42 2.2264554500579834
43 2.224668025970459
44 2.2228782176971436
45 2.2210848331451416
46 2.21928954124

In [38]:
_X = _X.cuda()
_y = _y.cuda()

In [39]:
pred = net(_X)

/home/cyclops/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [40]:
sum(pred.max(1)[1] == _y)


 229
[torch.cuda.ByteTensor of size () (GPU 0)]

In [41]:
loss_op(pred, _y)


 2.1070
[torch.cuda.FloatTensor of size () (GPU 0)]